In [3]:
import sys
sys.path.append('../')
import os
import json
import pandas as pd
import config

/app/nab
ROOT_PATH = /app


In [4]:
ROOT_RESULTS=config.NAB_ASSETS / "results"
SAVE_DIRECTORY= config.NAB_ASSETS / "processed_results"

In [5]:
#GET FOLDERS FOR EACH MODEL
folders=next(os.walk(ROOT_RESULTS))[1]

profiles=[("Standard","_standard_scores.csv"),("Low FN","_reward_low_FN_rate_scores.csv"),("Low FP","_reward_low_FP_rate_scores.csv")]

df_all_results=pd.DataFrame()
summary_results={}
for folder in folders: #for each model
    summary_results[folder]={}
    for name, file in profiles: #for each profile (name is profile name, file is the filename)
        summary_results[folder][name]={}
        filename=ROOT_RESULTS / folder / str(folder+file)
        df=pd.read_csv(filename).iloc[:-1,:] #remove last row that contained totals
    
        for item in ["Score","Total_Count","FP","FN","TP","TN"]:
            summary_results[folder][name][item]=df[item].sum()

        summary_results[folder][name]["threshold"]=df["Threshold"].values[0] #no sum as each treshold is the same
        summary_results[folder][name]["df"]=df

        #add to a dataframe that stores all results
        df_all_results=pd.concat([df_all_results,df], axis=0)

In [7]:
df_all_results

,Detector,Profile,File,Threshold,Score,TP,TN,FP,FN,Total_Count
0,bayesChangePt,standard,artificialNoAnomaly/art_daily_no_noise.csv,0.984615,-2.640000,0,3404,24,0,3428
1,bayesChangePt,standard,artificialNoAnomaly/art_daily_perfect_square_w...,0.984615,-2.640000,0,3404,24,0,3428
2,bayesChangePt,standard,artificialNoAnomaly/art_daily_small_noise.csv,0.984615,-2.420000,0,3406,22,0,3428
3,bayesChangePt,standard,artificialNoAnomaly/art_flatline.csv,0.984615,0.000000,0,3428,0,0,3428
4,bayesChangePt,standard,artificialNoAnomaly/art_noisy.csv,0.984615,0.000000,0,3428,0,0,3428
...,...,...,...,...,...,...,...,...,...,...
53,random,reward_low_FP_rate,realTweets/Twitter_volume_GOOG.csv,0.999595,-3.880000,0,13656,4,1432,15092
54,random,reward_low_FP_rate,realTweets/Twitter_volume_IBM.csv,0.999595,-2.880000,0,13549,4,1590,15143
55,random,reward_low_FP_rate,realTweets/Twitter_volume_KO.csv,0.999595,-1.696927,1,13511,3,1586,15101
56,random,reward_low_FP_rate,realTweets/Twitter_volume_PFE.csv,0.999595,-1.319839,2,13518,2,1586,15108


In [8]:
values=df_all_results.to_dict('records')

In [10]:
df_all_results
df_all_results[['Folder', 'Filename']] = df_all_results['File'].str.split('/', expand=True)
df_all_results.to_csv(SAVE_DIRECTORY / "all_datasets.csv",index=False)

In [11]:
#dataframe with summary results for each model / profile combo
totals=[]
for key,value in summary_results.items(): #for each model
    for p_key, p_value in summary_results[key].items(): #for each profile
        entry={}
        entry["model"]=key
        for item in ["Score","threshold","FP","FN","TP","TN"]:
            entry[item]=summary_results[key][p_key][item]
        entry["profile"]=p_key
        totals.append(entry)

df_totals=pd.DataFrame(totals)
df_totals.to_csv(SAVE_DIRECTORY / "new_model_results.csv")